# Pricing Asian Options Using Monte Carlo Control Variate

## Introduction

The payoff of an arithmetic Asian call option is defined as $$\Big(\frac{1}{N+1} \sum_{i=0}^{N}S_{t_i}-K \Big).$$
Calculating the price of this option using Monte Carlo simulation is straightforward, however achieving small standard errors requires a very large number of simulations and is thus a time consuming process.

An alternative solution is to use the payoff of a geometric Asian call option as the control variate: 
$$
\Big( \big( \prod_{i=0}^{N}S_{t_i} \big)^{\frac{1}{N+1}} \Big) ^{+}
$$

An analytical solution to the price of this options exists under GBM assumptions and this value along with the distance between MC simulations  would allow us to obtain an approximate for the analytical formula for the arithmetic Asian price.

We will assume that stock prices follow GBM. i.e.,
$$
S(\Delta t) = S_0 \exp \big[(\mu - \frac{\sigma^2}{2})\Delta t+(\sigma \sqrt{\Delta t})\epsilon \big]
$$

## Analytical Solution

The price of a geometric Asian option in the Black-Scholes model is given by:
$$
P_g = e^{-rT} \big( S_0 e^{\rho^T} N(d_1) - KN(d_2) \big)
$$
where $$ \rho = \frac{1}{2} \big( r-\frac{1}{2}\sigma^2 + \hat{\sigma}^2 \big)'$$
$$
\hat{\sigma}^2 := \sigma \frac{2N+1}{6N+1}
$$

with $\hat{\sigma}$ being the adjusted sigma and $N$ the total number of trading days ($T*252$)

$$d_1 = \frac{1}{\sqrt{T}\hat{\sigma}} \big( \ln(\frac{S_0}{K}) + (\rho+\frac{\hat{\sigma}}{2}) T \big) ;$$

$$d_2 = \frac{1}{\sqrt{T}\hat{\sigma}} \big( \ln(\frac{S_0}{K}) - (\rho+\frac{\hat{\sigma}}{2}) T \big) $$

We will start by pricing a geometric Asian call option using the above formula. Consider the following parameters: $ r = 3\%, \sigma = 0.3, S_0 = 100, K=100$, and $T=5$.


In [9]:
sighatt <- function(sig, N){
  sighat = sig*sqrt((2*N + 1)/(6*(N + 1)))
  return(sighat)
}
rhoo <- function(r, sig, sighat){
  rho=0.5*(r - 0.5*sig^2 + sighat^2)
  return(rho)
}
d11 <- function(tau, sighat, S0, K, rho){
  d1 = (1/(sighat*sqrt(tau)))*(log(S0/K) + (rho + 0.5*sighat^2)*tau)
 return(d1) 
}

d22 <- function(tau, sighat, S0, K, rho){
  d2 = (1/(sighat*sqrt(tau)))*(log(S0/K) + (rho - 0.5*sighat^2)*tau)
  return(d2) 
}

Pgg <- function(r, tau, S0, K, rho, d1, d2){
  Pg = exp(-r*tau)*(S0*exp(rho*tau)*pnorm(d1) - K*pnorm(d2))
  return(Pg)
}

r = 0.03
sig = 0.3
S0 = 100
K = 100
tau = 5
N = tau*252
q = 0
sighat = sighatt(sig,N)
rho = rhoo(r,sig,sighat)
d1 = d11(tau,sighat,S0,K,rho)
d2 = d22(tau,sighat,S0,K,rho)
Pg = Pgg(r,tau,S0,K,rho,d1,d2)
print(Pg)

[1] 15.17113


## Arithmetic Asian Option Using Monte Carlo

Now, we will implement a Monte Carlo scheme to price an arithmetic Asian call option with 1,000,000 simulations. We will also record the confidence interval and the time it takes to obtain the result.

In [8]:
set.seed(23)
lnS = log(S0)
AMC <- function(m, n){
  matr <- matrix(NA,n+1,2)
  Xi = matrix(NA,m,1)
  dt = tau/n
  matr[1,] = c(0,S0)
  nudt = (r-q-0.5*sig^2)*dt
  sigsdt = sig*sqrt(dt)
  lnS = log(S0)
  Sum_CT = 0
  Sum_CT2 = 0
  ptm <- proc.time()
  for(j in 1:m) {
    lnSt = lnS
    for(i in 1:n) {
      lnSt = lnSt + nudt + sigsdt*rnorm(1)
      matr[i + 1, 1] = i*dt
      matr[i + 1, 2] = exp(lnSt)
    }
    CT = pmax((1/(n + 1))*sum(matr[, 2]) - K, 0)
    Xi[j, 1]=CT*exp(-r*tau)
    Sum_CT = Sum_CT + CT
    Sum_CT2 = Sum_CT2 + CT*CT
  }
  assign("X", Xi, .GlobalEnv)
  price = Sum_CT/m*exp(-r*tau)
  SD = sqrt((Sum_CT2 - Sum_CT*Sum_CT/m)*exp(-2*r*tau)/(m - 1))
  chrono <- (proc.time() - ptm)[3]
  newlist <- list("price"=price, "SD" = SD, "time" = chrono)
  return(newlist)
}

#AMCC <- AMC(1000000, N)
#Pa <- AMCC$price
#print(AMCC)

The result obtained is $P_{a}^{sim}$=17.50066. Given the standard deviation is 0.3078367, the 95% confidence interval for this solution is $[17.44066, 17.56066]$.

## Geometric Asian Option Using Monte Carlo


We do the same for a Geometric Asian option with the same parameters.

In [11]:
set.seed(23)
lnS = log(S0)
GMC <- function(m, n){
  matr <- matrix(NA, n + 1, 2)
  Yi = matrix(NA, m, 1)
  dt = tau/n
  matr[1,] = c(0,S0)
  nudt = (r-q-0.5*sig^2)*dt
  sigsdt = sig*sqrt(dt)
  lnS = log(S0)
  Sum_CT = 0
  Sum_CT2 = 0
  ptm <- proc.time()
  for(j in 1:m) {
    lnSt = lnS
    for(i in 1:n) {
      lnSt = lnSt + nudt + sigsdt*rnorm(1)
      matr[i+1,1]=i*dt
      matr[i+1,2]=lnSt
    }
    Prod = exp(mean(matr[,2]))
    #Prod=exp(sum(matr[,2]))/(n+1)
    CT = pmax(Prod - K,0)
    Yi[j, 1] = CT*exp(-r*tau)
    Sum_CT = Sum_CT+CT
    Sum_CT2 = Sum_CT2+CT*CT
  }
  assign("Y", Yi,.GlobalEnv)
  price = Sum_CT/m*exp(-r*tau)
  SD = sqrt((Sum_CT2-Sum_CT*Sum_CT/m)*exp(-2*r*tau)/(m - 1))
  chrono <- (proc.time() - ptm)[3]
  newlist <- list("price" = price, "SD" = SD, "time" = chrono)
  return(newlist)
}
#  GMCC <- (GMC(200000, N))
#  print(GMCC)
#  err <- GMCC$price


$price
[1] 54.07486

$SD
[1] 58.12661

$time
elapsed 
690.311 

$price
[1] 65.39111

$SD
[1] 61.5649

$time
elapsed 
676.539 



The result obtained is $P_{g}^{sim}$=15.35956. Given the standard deviation is 0.3067162, the 95% confidence interval for this solution is $[15.29956, 15.41956]$.

## Control Variate

Rather than running 1 million simulations, let us run only $M=$10,000 paths for each option and assign
- item numbers $X_i$ which are M replications for the arithmetic Asian Option price
- item numbers $Y_i$ which are M replication for the geometric Asian Option price
to calculate $b^*$ such that

$$
b^* = \frac{\sum_{M=1}^{M}(X_i-\bar{X})(Y_i-\bar{Y})}{\sum_{M=1}^{M}(X_i-\bar{X})^2}
$$

i.e., $b^*$ is the slope of a regression line $Y = a + bX + \epsilon$.

> Note that this is already implemented in the code in the previous sections.

In [13]:
 z <- lm(X ~ Y)
 b <- as.double(z$coef[2])
b

[1] 0.004990768

We can Calculate the error of pricing for the geometric Asian: $E_g=P_g-P_g^{sim}$

In [15]:
errg <- Pg - err
errg

[1] -50.21998

And we can calculate the modified arithmetic option price ($P_a^*$) as $$P_a^*=P_a^{sim}-b^* E_g$$

In [18]:
Pmod <- AMCC$price - b*errg
Pmod

[1] 54.3255

We can see that while the second method is more than 90 times quicker than MonteCarlo without control variate the error is less than one percent.  